In [4]:
#coding=utf-8
import time
import datetime
import numpy as np
import pandas as pd
import lightgbm as lgb
from dateutil.parser import parse
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error
from dateutil.parser import parse

D:\02_Tools\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
def change(name):
    df = pd.read_csv(name,encoding='gb2312')
    index = ['尿素','*r-谷氨酰基转换酶','尿酸','*碱性磷酸酶','红细胞计数','血小板计数','血小板比积',
         '嗜酸细胞%','红细胞平均体积','红细胞体积分布宽度','红细胞平均血红蛋白浓度']
    for i in index:
        # index.remove(i)
        for j in index:
            if i == j:
                continue
            else:
                df['{}/{}'.format(i,j)] = df[i]/df[j]
    return df

In [6]:
df1 = change('d_train_20180102.csv')
print(df1.shape)
df1.to_csv('train_change.csv', index=False)

df2 = change('d_test_A_20180102.csv')
df2.to_csv('test_change.csv', index=False)

df3 = change('d_test_B_20180128.csv')
df3.to_csv('test_change_B.csv', index=False)

df4 = change('d_train_20180102_add.csv')
print(df4.shape)
df4.to_csv('train_change_add.csv', index=False)

(5642, 152)
(6642, 152)


In [7]:
train = pd.read_csv('train_change_add.csv')
test = pd.read_csv('test_change_B.csv')

In [8]:
# ol = pd.read_csv('d_answer_b_20180130.csv', encoding='utf-8')

In [9]:
# train.sort_index(by=['血糖'])
# train = train.loc[train['血糖']<36]
traincopy = train.copy()
testcopy = test.copy()
train["血糖"] = np.log1p(train["血糖"]) #先对血糖进行log（x+1）处理

In [10]:
# train_over_zero = train.loc[train['血糖']>=0]
# train_under_zero = train.loc[train['血糖']<0]

def make_feat(train, test):
    train_id = train.id.values.copy()
    test_id = test.id.values.copy()
    data = pd.concat([train, test])

    data['性别'] = data['性别'].map({'男': 1, '女': 0})
    data['体检日期'] = (pd.to_datetime(data['体检日期']) - parse('2017-10-09')).dt.days

    data.fillna(data.mean(axis=0), inplace=True)

    train_feat = data[data.id.isin(train_id)]
    test_feat = data[data.id.isin(test_id)]

    return train_feat, test_feat

In [11]:
train_feat, test_feat = make_feat(train, test)
# train_feat_over, test_feat_over = make_feat(train_over_zero, test)
# train_feat_under, test_feat_under = make_feat(train_under_zero, test)
train_feat_copy, test_feat_copy = make_feat(traincopy, testcopy)

D:\02_Tools\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [12]:
train_feat.drop(['id'], axis=1, inplace=True)
test_feat.drop(['id'], axis=1, inplace=True)

# test_feat.drop(['体检日期' , '乙肝表面抗原' ,  '乙肝e抗原' , '乙肝e抗体'
#                   , '乙肝核心抗体'], axis=1, inplace=True)

In [13]:
predictors = [f for f in test_feat.columns if f not in ['血糖']]


In [14]:
def evalerror(pred, df):
    label = df.get_label().values.copy()
    score = mean_squared_error(label, pred) * 0.5
    return ('0.5mse', score, False)

In [15]:
print('开始训练...')
params = {
    'learning_rate': 0.01,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mse',
    'sub_feature': 0.8,
    'num_leaves': 60,
    'colsample_bytree': 0.7,
    'feature_fraction': 0.7,
    'min_data': 100,
    'min_hessian': 1,
    'verbose': -1,
}

开始训练...


In [16]:
def lgb_train(train_feat,test_feat):
    print('开始CV 5折训练...')
    t0 = time.time()
    train_preds = np.zeros(train_feat.shape[0])
    test_preds = np.zeros((test_feat.shape[0], 5))
    kf = KFold(len(train_feat), n_folds=5, shuffle=True, random_state=500)
    for i, (train_index, test_index) in enumerate(kf):
        print('第{}次训练...'.format(i))
        train_feat1 = train_feat.iloc[train_index]
        train_feat2 = train_feat.iloc[test_index]
        # lgb_train1 = lgb.Dataset(train_feat1[predictors], train_feat1['血糖'], categorical_feature=['性别'])
        # lgb_train2 = lgb.Dataset(train_feat2[predictors], train_feat2['血糖'])
        # gbm = lgb.train(params,
        #                 lgb_train1,
        #                 num_boost_round=3000,
        #                 valid_sets=lgb_train2,
        #                 verbose_eval=None,
        #                 feval=evalerror,
        #                 early_stopping_rounds=110)

        model_lgb = lgb.LGBMRegressor(objective='regression', num_leaves=8,
                                      learning_rate=0.05, n_estimators=400,
                                      max_bin=30, bagging_fraction=0.9,
                                      bagging_freq=10, feature_fraction=0.5,
                                      feature_fraction_seed=10, bagging_seed=10,
                                      min_data_in_leaf=80, nthread=8,
                                      min_sum_hessian_in_leaf=0.2)
        gbm = model_lgb.fit(train_feat1[predictors], train_feat1['血糖'], categorical_feature=['性别'])
        train_preds[test_index] += gbm.predict(train_feat2[predictors])
        test_preds[:, i] = gbm.predict(test_feat[predictors])
    return train_preds,test_preds

In [17]:
train_preds,test_preds = lgb_train(train_feat,test_feat)
train_preds = np.expm1(train_preds)
test_preds = np.expm1(test_preds)  #血糖值回算：exp（x）-1

开始CV 5折训练...
第0次训练...


D:\02_Tools\Anaconda3\lib\site-packages\lightgbm\basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['性别']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


第1次训练...


D:\02_Tools\Anaconda3\lib\site-packages\lightgbm\basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['性别']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


第2次训练...


D:\02_Tools\Anaconda3\lib\site-packages\lightgbm\basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['性别']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


第3次训练...


D:\02_Tools\Anaconda3\lib\site-packages\lightgbm\basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['性别']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


第4次训练...


D:\02_Tools\Anaconda3\lib\site-packages\lightgbm\basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['性别']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


In [20]:
#对预测值大于6.4的血糖进行后处理
for i in range(len(train_preds)):
    if train_preds[i] >= 6.4:
        # train_preds[i] = ((train_preds[i]-6.4)*1.4)**1.26 +6.4
        train_preds[i] = np.expm1((train_preds[i] - 6.4) * 0.8)*0.45 + train_preds[i]

test_preds1 = test_preds.mean(axis=1)

In [22]:
ol = pd.read_csv('d_answer_b_20180130.csv', encoding='gb2312')

In [23]:
for i in range(len(test_preds1)):
    if test_preds1[i] >= 6.4:
        # test_preds1[i] = ((test_preds1[i]-6.4)*1.4)**1.26 + 6.4
        test_preds1[i] = np.expm1((test_preds1[i] - 6.4) * 0.8)*0.45 + test_preds1[i]

print('线下得分：    {}'.format(mean_squared_error(train_feat_copy['血糖'], train_preds) * 0.5))
# print(test_preds1[5])
# print('CV训练用时{}秒'.format(time.time() - t0))

print('线上得分：    {}'.format(mean_squared_error(ol['血糖'], test_preds1) * 0.5))

线下得分：    924698.229727483
线上得分：    0.6234388778898579


In [24]:
submission = pd.DataFrame({'pred': test_preds1})
if mean_squared_error(train_feat_copy['血糖'], train_preds) * 0.5 < 1.2:
    submission.to_csv(r'sub{}_{}.csv'.format(round(mean_squared_error(train_feat_copy['血糖'], train_preds) * 0.5,3),
    datetime.datetime.now().strftime('%Y%m%d_%H%M%S')), header=None,index=False, float_format='%.4f')

In [ ]:
# test
